In [1]:
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce

#import matplotlib.pyplot as plt

import cProfile, pstats, io
from pstats import SortKey

torch.manual_seed(0)

In [2]:
import numpy as np
ids=get_ids_in_folder(base_path="./data/public/1.0.1/test/")

idx=10
data=load_npz_by_id(idx,base_path="./data/public/1.0.1/test/")



#data=np.load("path")
tmp_name=data["alos"][idx] #shape : 5000;4;4;4;3 (batch / time / h / w / c)
print(tmp_name.shape)

#3456
#11520
#1920
#420
#192



(4, 4, 4, 3)


In [3]:
#compute_channel_mean_std(mode="train")

In [4]:
#create_dataset(name="tiny", mode="train",max_imgs=-1)
#create_dataset(name="tiny", mode="test",max_imgs=-1)
#create_dataset(name="tiny", mode="validation",max_imgs=-1)

In [5]:
config=read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
transconfig=transformations_config_flair("./data/bands_info/bands.yaml",config)

dataset=CustomPlanted("./data/custom_planted/tiny_train.h5", config=config, trans_config=transconfig)

In [6]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torch
import os

seed_everything(42, workers=True)

torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')

config_model = read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
labels = load_json_to_dict("./data/labels.json")
bands_yaml = "./data/bands_info/bands.yaml"

trans_config = transformations_config_flair(bands_yaml, config_model)
xp_name = "test_Atos"

wand = False
wandb_logger = None
if wand:
    if os.environ.get("LOCAL_RANK", "0") == "0":
        import wandb
        wandb.init(
            name=get_xp_name(config_model['encoder']) + " modalities",
            project="Atos_modalities",
            config=config_model
        )
        wandb_logger = WandbLogger(project="tiny_modalities")

model = Model(config_model, wand=wand, name=xp_name, labels=labels,transform=trans_config)

data_module = CustomPlantedDataModule(
    "./data/custom_planted/tiny",
    config=config_model,
    trans_config=trans_config,
    batch_size=config_model['dataset']['batchsize'],
)

data_module.setup()


# Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints/",
    filename="best_model-{epoch:02d}-{val_F1:.4f}",
    monitor="val_F1",
    mode="max",
    save_top_k=1,
    verbose=True,
)

early_stop_callback = EarlyStopping(
    monitor="val_F1",
    min_delta=0.00,
    patience=15,
    verbose=False,
    mode="max"
)



# Trainer
trainer = Trainer(
    use_distributed_sampler=False,
    #strategy="ddp",
    #devices=-1,
    max_epochs=config_model["trainer"]["epochs"],
    logger=wandb_logger,
    log_every_n_steps=1,
    accelerator="gpu",
    callbacks=[early_stop_callback, checkpoint_callback],
    default_root_dir="./checkpoints/",
    limit_train_batches=0.001,
    limit_val_batches=0.001
)

pr = cProfile.Profile()
pr.enable()
# Fit the model
trainer.fit(model, datamodule=data_module)

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())


Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/hugoriffaud/Documents/PLANTED/checkpoints exists and is not empty.

   | Name                      | Type                      | Params | Mode 
-------------------

tensor([0.9906, 1.3819, 0.0697, 0.1518, 0.5864, 0.7594, 0.2928, 0.4033, 1.2501,
        1.6072, 0.8545, 0.6561, 0.5780, 0.2378, 0.4709, 1.0382, 0.6284, 1.7651,
        1.9440, 1.4457, 4.0108, 3.9630, 1.6251, 6.4757, 3.4562, 3.9920, 2.8993,
        3.7461, 3.1281, 5.1480, 4.7641, 4.0511, 4.3674, 5.6088, 5.5168, 7.2073,
        5.7102, 6.4951, 6.4380, 7.7178])
Sanity Checking: |          | 0/? [00:00<?, ?it/s]Validation DataLoader created on rank: 0


/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 14 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Train DataLoader created on rank: 0                                        


/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


...

Epoch 3: 100%|██████████| 67/67 [01:24<00:00,  0.79it/s, v_num=211]

Epoch 3, global step 268: 'val_F1' reached 0.07832 (best 0.07832), saving model to '/Users/hugoriffaud/Documents/PLANTED/checkpoints/best_model-epoch=03-val_F1=0.0783.ckpt' as top 1


Epoch 4: 100%|██████████| 67/67 [01:22<00:00,  0.82it/s, v_num=211]

Epoch 4, global step 335: 'val_F1' reached 0.10108 (best 0.10108), saving model to '/Users/hugoriffaud/Documents/PLANTED/checkpoints/best_model-epoch=04-val_F1=0.1011.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 67/67 [01:22<00:00,  0.81it/s, v_num=211]
         6379025 function calls (5972537 primitive calls) in 392.709 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  392.712  196.356 /Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3513(run_code)
      7/2    0.000    0.000  392.712  196.356 {built-in method builtins.exec}
        1    0.000    0.000  392.712  392.712 /Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py:507(fit)
        1    0.000    0.000  392.712  392.712 /Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:35(_call_and_handle_interrupt)
        1    0.000    0.000  392.712  392.712 /Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py:565(_fit